In [1]:
#!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.1 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [ ]:
# scen_gen_ext.py  -------------------------------------------------
import numpy as np
from scipy.stats import norm, beta, weibull_min
from scipy.special import gamma


class scen_gen:
    def __init__(self, sample_size, time_period,
                 solar_number, load_number, wind_number,
                 seed: int | None = 123):
        self.S  = sample_size
        self.T  = time_period
        self.ns = solar_number
        self.nl = load_number
        self.nw = wind_number

        # single global RNG for reproducible yet different draws per hour
        self.rng = np.random.default_rng(seed)

    # ---------------------------- Load ---------------------------
    def load_scen_gen(self, load_forecast: np.ndarray):
        μ = load_forecast
        σ = 0.03 * μ

        Load_Power = []
        for t in range(self.T):
            q = self.rng.random((self.S, self.nl))
            Load_Power.append(norm.ppf(q, loc=μ[t], scale=σ[t]))
        return Load_Power

    # ---------------------------- Solar --------------------------
    def solar_irr_scen_gen(self, irr_forecast: np.ndarray):
        μ = irr_forecast / 1000.0              # scale to (0,1)
        μ = np.clip(μ, 1e-9, 0.99)             # avoid α,β negatives

        σ = 0.03 * μ
        α = μ**2 * ((1 - μ) / σ**2 - 1 / μ)
        β = α * (1 / μ - 1)

        sol_scen = []
        for t in range(self.T):
            q = self.rng.random((self.S, self.ns))
            sol_scen.append(beta.ppf(q, α[t], β[t]) * 1000.0)  # back to W/m²
        return sol_scen

    # ---------------------------- Wind ---------------------------
    @staticmethod
    def _fit_weibull(mean, std):
        k = (std / mean) ** -1.086
        lam = mean / gamma(1 + 1 / k)
        return k, lam

    def wind_speed_scen_gen(self, wind_forecast: np.ndarray):
        μ = wind_forecast
        σ = 0.03 * μ

        Wind_Speed = []
        for t in range(self.T):
            k, lam = self._fit_weibull(μ[t], σ[t])
            q = self.rng.random((self.S, self.nw))
            Wind_Speed.append(weibull_min.ppf(q, c=k, scale=lam))
        return Wind_Speed

In [ ]:
# scen_red_bwd.py  -------------------------------------------------
import numpy as np


class scen_red:
    def __init__(self, generated_samples, sample_size,
                 number_desired_scenarios, time_period,
                 load_number, solar_number, wind_number):
        # deep-copy so caller’s list isn’t mutated
        self.generated_samples_copy = [arr.copy() for arr in generated_samples]

        self.sample_size  = sample_size          # S_init
        self.reduced_size = number_desired_scenarios
        self.time_period  = time_period

        self.load_number  = load_number
        self.solar_number = solar_number
        self.wind_number  = wind_number

    # -------------------------------------------------------------
    def _scaled_distance_matrix(self, A: np.ndarray) -> np.ndarray:
        scale = np.std(A, axis=0, ddof=1)
        scale[scale < 1e-9] = 1.0      # avoid division by ~0
        A_scaled = A / scale

        # Vectorised pair-wise || a_i - a_j ||₂
        d = np.linalg.norm(A_scaled[:, None, :] - A_scaled[None, :, :], axis=2)
        np.fill_diagonal(d, np.inf)
        return d

    # -------------------------------------------------------------
    def reduced_scen(self):
        # -------- Step-0: initial equal probabilities -------------
        prob_matrix = [np.ones(self.sample_size) / self.sample_size
                       for _ in range(self.time_period)]

        # -------- Backward elimination loop ----------------------
        while self.sample_size > self.reduced_size:
            for t in range(self.time_period):

                A = self.generated_samples_copy[t]          # (S, n_vars)
                dist = self._scaled_distance_matrix(A)      # (S, S)

                # 1) closest neighbour & min distance per column
                min_dist  = dist.min(axis=0)
                min_index = dist.argmin(axis=0)

                # 2) cost = probability × distance
                cost   = min_dist * prob_matrix[t]
                victim = cost.argmin()          # scenario to remove
                partner = min_index[victim]     # its nearest neighbour

                # 3) choose survivor vs. victim (keep higher-prob one)
                if prob_matrix[t][partner] >= prob_matrix[t][victim]:
                    survivor, delete = partner, victim
                else:
                    survivor, delete = victim, partner

                # 4) merge probability and delete scenario
                prob_matrix[t][survivor] += prob_matrix[t][delete]
                prob_matrix[t] = np.delete(prob_matrix[t], delete)
                self.generated_samples_copy[t] = np.delete(
                    self.generated_samples_copy[t], delete, axis=0)

            self.sample_size -= 1      # one scenario removed per outer pass

        # -------- Split reduced samples into variable blocks -----
        reduced = self.generated_samples_copy
        Load_Scen, Solar_Scen, Wind_Scen = [], [], []

        for t in range(self.time_period):
            Load_Scen.append(reduced[t][:, :self.load_number])

            idx_solar_end = self.load_number + self.solar_number
            Solar_Scen.append(reduced[t][:, self.load_number:idx_solar_end])

            Wind_Scen.append(reduced[t][:, idx_solar_end:])

        return (reduced, prob_matrix,
                Load_Scen, Solar_Scen, Wind_Scen)

In [18]:
# =================================================================
#  Scenario-Based Day-Ahead MILP  – 30-Bus, 9-MGC, 21-MG  (DATA BLOCK)
#  -----------------------------------------------------------------
#  This block supersedes the deterministic 30-bus parameter section
#  when building the stochastic version.  All names/structures match
#  the deterministic script so the later adaptation is drop-in.
# =================================================================
from mip import Model, xsum, BINARY, MINIMIZE, MAXIMIZE, OptimizationStatus
import numpy as np
import pandas as pd

# --------------------------------------------------
# 1. GLOBAL SETTINGS
# --------------------------------------------------
T          = 24                    # hourly horizon
NUM_MGC    = 9                     # controllers on buses 1-2-3-6-28-13-23-22-27
NUM_MG     = 21                    # microgrids on buses 4-…-30
Delta_t    = 1.0                   # h
M          = 1e3                   # Big-M

# Index sets (keep labels identical to deterministic version)
MGC        = list(range(NUM_MGC))
MG         = list(range(NUM_MG))

PV_MGs     = list(range(0, 6))         # MG-0…5  (PV + battery)
WIND_MGCs  = MGC                       # wind turbine on every MGC
WIND_MGs   = list(range(6, NUM_MG))    # MG-6…20 wind sites
BATT_MG    = PV_MGs[:]                 # batteries co-located with PV MGs

# --------------------------------------------------
# 2. MGC PARAMETERS  (copied from deterministic 30-bus model)
# --------------------------------------------------
P_MT_MGC_MIN  = [0] * NUM_MGC
P_MT_MGC_MAX  = [6.0, 6.0, 5.5, 5.0, 6.5, 5.5, 5.5, 5.0, 5.0]
RU_MGC        = [3.0, 3.0, 2.8, 2.5, 3.2, 3.0, 3.0, 2.6, 2.6]
RD_MGC        = RU_MGC[:]
MIN_UP        = [2] * NUM_MGC
MIN_DOWN      = [2] * NUM_MGC
STARTUP_COST  = [2.2, 2.3, 2.1, 2.0, 2.4, 2.2, 2.3, 2.1, 2.0]
SHUTDOWN_COST = [0.7, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.6]
ETA_MGC       = [0.42, 0.43, 0.41, 0.40, 0.44, 0.42, 0.42, 0.41, 0.40]
C_NG_MGC      = [15.0, 14.8, 14.5, 14.5, 15.2, 14.9, 15.0, 14.7, 14.7]
L_NG_MGC      = [3] * NUM_MGC
P_GRID_MAX    = [20] * NUM_MGC

MARKUP        = [1.10] * NUM_MGC     # 10 % retail margin
C_GRID = [                           # 24-h wholesale price ($/MWh)
    50.61, 49.88, 49.41, 47.86, 46.07, 45.07, 45.68, 47.44,
    49.44, 49.78, 46.09, 45.60, 45.04, 43.54, 35.85, 43.52,
    44.41, 49.00, 52.22, 52.39, 52.91, 53.42, 52.97, 51.51
]
C_EXC = [[round(p * MARKUP[i], 2) for p in C_GRID] for i in MGC]

P_MGC_MAX   = 35          # MW cap per MG for imported power from all MGCs
LAMBDA_ATC  = 0.5         # redispatch penalty coefficient

# Wind curtailment penalty
C_WIND = [0.0005] * (len(WIND_MGCs) + len(WIND_MGs))

# --------------------------------------------------
# 3. LOCAL-MG MICRO-TURBINES  (two GTs per MG)
# --------------------------------------------------
C_NG   = 4.0
L_NG   = 4.8
RU_MG  = 2
RD_MG  = 2

# Min / max output per GT (21×2)
P_MG_MIN = [
    [2, 2],   [2, 2],   [2, 2],   [1.8, 1.8], [1.8, 1.8], [2, 2],      # MG-0…5
    [2.5, 2.5], [2, 2], [2.2, 2.2], [2, 2],   [2.2, 2.2], [2, 2],
    [2, 2],   [2, 2],                                               # MG-6…13
    [0, 0],   [0, 0],   [0, 0],   [0, 0],   [0, 0],   [0, 0], [0, 0]   # MG-14…20
]
P_MG_MAX = [
    [6, 6],   [6, 6],   [6, 6],   [5.5, 5.5], [5.5, 5.5], [6, 6],
    [8, 8],   [7, 7],   [7.5, 7.5], [7, 7],   [7.5, 7.5], [7, 7],
    [7, 7],   [7, 7],
    [0, 0],   [0, 0],   [0, 0],   [0, 0],   [0, 0],   [0, 0], [0, 0]
]
ETA_MG = [
    [0.30, 0.35] if any(P_MG_MAX[i]) else [0, 0]
    for i in range(NUM_MG)
]

# --------------------------------------------------
# 4. RENEWABLE-SITE PARAMETERS
# --------------------------------------------------
# PV: MG-0…5
PV_PARAMS_MG = {
    0: dict(G_thr=55,  G_std=1000, P_rated=3.2),
    1: dict(G_thr=50,  G_std=1000, P_rated=3.0),
    2: dict(G_thr=48,  G_std=1000, P_rated=2.8),
    3: dict(G_thr=45,  G_std=1000, P_rated=2.6),
    4: dict(G_thr=43,  G_std=1000, P_rated=2.4),
    5: dict(G_thr=40,  G_std=1000, P_rated=2.2),
}

# Wind at all MGC sites
WIND_PARAMS_MGC = {
    0: dict(v_ci=3.0, v_r=11.0, v_co=25.0, P_rated=10),
    1: dict(v_ci=3.2, v_r=11.5, v_co=25.0, P_rated=11),
    2: dict(v_ci=3.4, v_r=12.0, v_co=25.0, P_rated=9),
    3: dict(v_ci=3.1, v_r=11.2, v_co=25.0, P_rated=12),
    4: dict(v_ci=3.3, v_r=12.5, v_co=25.0, P_rated=10),
    5: dict(v_ci=3.0, v_r=11.0, v_co=25.0, P_rated=11),
    6: dict(v_ci=3.2, v_r=12.0, v_co=25.0, P_rated=10),
    7: dict(v_ci=3.1, v_r=11.5, v_co=25.0, P_rated=9),
    8: dict(v_ci=3.4, v_r=12.5, v_co=25.0, P_rated=12),
}
# Wind at MG-6…20
WIND_PARAMS_MG = {
     6: dict(v_ci=2.0, v_r=8.0,  v_co=11.0, P_rated=5),
     7: dict(v_ci=2.1, v_r=8.2,  v_co=11.0, P_rated=6),
     8: dict(v_ci=2.2, v_r=8.5,  v_co=11.0, P_rated=5.5),
     9: dict(v_ci=2.0, v_r=7.8,  v_co=11.0, P_rated=5),
    10: dict(v_ci=2.3, v_r=8.3,  v_co=11.0, P_rated=6),
    11: dict(v_ci=2.2, v_r=8.1,  v_co=11.0, P_rated=5.5),
    12: dict(v_ci=2.1, v_r=8.0,  v_co=11.0, P_rated=6),
    13: dict(v_ci=2.4, v_r=8.4,  v_co=11.0, P_rated=6.5),
    14: dict(v_ci=2.0, v_r=7.5,  v_co=11.0, P_rated=6),
    15: dict(v_ci=2.2, v_r=7.8,  v_co=11.0, P_rated=6.5),
    16: dict(v_ci=2.1, v_r=7.6,  v_co=11.0, P_rated=6),
    17: dict(v_ci=2.3, v_r=7.9,  v_co=11.0, P_rated=6.5),
    18: dict(v_ci=2.0, v_r=7.5,  v_co=11.0, P_rated=6),
    19: dict(v_ci=2.2, v_r=7.8,  v_co=11.0, P_rated=6.5),
    20: dict(v_ci=2.1, v_r=7.6,  v_co=11.0, P_rated=6),
}

# --------------------------------------------------
# 5. BATTERY PARAMETERS  (MG-0…5)
# --------------------------------------------------
E_R        = [0.30, 0.32, 0.28, 0.26, 0.25, 0.27]
E_init     = [0.07, 0.08, 0.06, 0.06, 0.06, 0.07]
E_end      = [0.07, 0.08, 0.06, 0.06, 0.06, 0.07]
P_chr_max  = [0.06, 0.06, 0.05, 0.05, 0.05, 0.05]
P_dis_max  = [0.06, 0.06, 0.05, 0.05, 0.05, 0.05]
SOC_min    = [0.20] * len(BATT_MG)
SOC_max    = [0.80] * len(BATT_MG)
ETA_CHR    = [0.92] * len(BATT_MG)
ETA_DIS    = [0.92] * len(BATT_MG)
K_CHR, K_DIS = 0.002, 0.001

# --------------------------------------------------
# 6. TRANSMISSION LINES (GSF/PTDF)
LINES = {
    'L12'   : {'limit': 20, 'gsf': [ 0.637187, -0.20191, 0.12766, 0.020392, -0.116623,	-0.031336,	-0.065451,	-0.031213,	-0.022587,	-0.018004,	-0.022587,	-0.000638,	-0.000638,	-0.003155,	-0.005091,	-0.008028,	-0.015048,	-0.0096,	-0.012265,	-0.0137,	-0.017653,	-0.017552,	-0.009779,	-0.016108,	-0.021194,	-0.021194,	-0.024431,	-0.030596,	-0.024431,	-0.024431]},
    'L13'   : {'limit': 50, 'gsf': [ 0.329479,	0.168576,	-0.160994,	-0.053725,	0.083289,	-0.001998,	0.032117,	-0.002121,	-0.010747,	-0.01533,	-0.010747,	-0.032695,	-0.032695,	-0.030178,	-0.028242,	-0.025306,	-0.018285,	-0.023733,	-0.021068,	-0.019634,	-0.015681,	-0.015781,	-0.023554,	-0.017226,	-0.012139,	-0.012139,	-0.008903,	-0.002737,	-0.008903,	-0.008903]},
    'L24'   : {'limit': 20, 'gsf': [ 0.213034,	0.291493,	-0.035421,	-0.087727,	0.146003,	0.000514,	0.05871,	0.000304,	-0.014411,	-0.022229,	-0.014411,	-0.051853,	-0.051853,	-0.047559,	-0.044256,	-0.039247,	-0.027271,	-0.036564,	-0.032019,	-0.029571,	-0.022828,	-0.022999,	-0.036259,	-0.025463,	-0.016787,	-0.016787,	-0.011265,	-0.000748,	-0.011265,	-0.011265]},
    'L34'   : {'limit': 20, 'gsf': [ 0.296146,	0.135243,	0.805673,	-0.087058,	0.049956,	-0.035331,	-0.001216,	-0.035454,	-0.04408,	-0.048663,	-0.04408,	-0.066029,	-0.066029,	-0.063512,	-0.061575,	-0.058639,	-0.051619,	-0.057066,	-0.054402,	-0.052967,	-0.049014,	-0.049114,	-0.056888,	-0.050559,	-0.045473,	-0.045473,	-0.042236,	-0.036071,	-0.042236,	-0.042236]},
    'L25'   : {'limit': 20, 'gsf': [ 0.137381,	0.162967,	0.056358,	0.039301,	-0.420585,	-0.004137,	-0.170716,	-0.004034,	0.00321,	0.007058,	0.00321,	0.021641,	0.021641,	0.019528,	0.017902,	0.015436,	0.009541,	0.014115,	0.011878,	0.010673,	0.007353,	0.007438,	0.013965,	0.008651,	0.004379,	0.004379,	0.001661,	-0.003516,	0.001661,	0.001661]},
    'L26'   : {'limit':  20, 'gsf': [ 0.253439,	0.310297,	0.073389,	0.035484,	0.124626,	-0.061046,	0.013223,	-0.060816,	-0.044719,	-0.036167,	-0.044719,	-0.00376,	-0.00376,	-0.008457,	-0.01207,	-0.01755,	-0.030651,	-0.020485,	-0.025457,	-0.028134,	-0.035511,	-0.035324,	-0.020818,	-0.032628,	-0.04212,	-0.04212,	-0.04816,	-0.059665,	-0.04816,	-0.04816]},
    'L46'   : {'limit': 20, 'gsf': [0.235083,	0.157492,	0.480788,	0.532516,	-0.059699,	-0.276891,	-0.190015,	-0.274962,	-0.139988,	-0.068277,	-0.139988,	0.203453,	0.203453,	0.164068,	0.133773,	0.087823,	-0.022025,	0.063216,	0.021523,	-0.000927,	-0.062782,	-0.061212,	0.06042,	-0.038607,	-0.118196,	-0.118196,	-0.168843,	-0.265315,	-0.168843,	-0.168843]},
    'L57'   : {'limit':  20, 'gsf': [ 0.104048,	0.129634,	0.023025,	0.005968,	0.546082,	-0.037471,	-0.20405,	-0.037367,	-0.030123,	-0.026275,	-0.030123,	-0.011692,	-0.011692,	-0.013806,	-0.015432,	-0.017898,	-0.023793,	-0.019218,	-0.021456,	-0.02266,	-0.02598,	-0.025896,	-0.019368,	-0.024683,	-0.028954,	-0.028954,	-0.031672,	-0.036849,	-0.031672,	-0.031672]},
    'L67'   : {'limit':  20, 'gsf': [ -0.070714,	-0.0963,	0.010308,	0.027366,	-0.512748,	0.070804,	-0.762617,	0.0707,	0.063457,	0.059608,	0.063457,	0.045025,	0.045025,	0.047139,	0.048765,	0.051231,	0.057126,	0.052551,	0.054789,	0.055994,	0.059313,	0.059229,	0.052702,	0.058016,	0.062287,	0.062287,	0.065005,	0.070183,	0.065005,	0.065005]},
    'L68'   : {'limit': 20, 'gsf': [ 0.065404,	0.065589,	0.064818,	0.064695,	0.066107,	0.066624,	0.066417,	-0.79879,	0.053269,	0.046274,	0.053269,	0.049985,	0.049985,	0.047196,	0.045051,	0.048406,	0.046906,	0.045478,	0.04573,	0.045866,	0.041641,	0.040318,	0.034926,	0.021258,	-0.030391,	-0.030391,	-0.063259,	-0.125863,	-0.063259,	-0.063259]},
    'L69' : {'limit':  50, 'gsf': [ 0.205603,	0.208103,	0.197687,	0.196021,	0.2151,	0.222098,	0.219299,	0.219554,	-0.394587,	-0.193803,	-0.394587,	-0.00279,	-0.00279,	-0.023313,	-0.0391,	-0.084072,	-0.16129,	-0.093123,	-0.125046,	-0.142236,	-0.174962,	-0.169579,	-0.061636,	-0.092061,	0.012871,	0.012871,	0.079645,	0.206835,	0.079645,	0.079645]},
    'L6_10' : {'limit':  20, 'gsf': [ 0.104392,	0.105821,	0.099869,	0.098917,	0.109819,	0.113818,	0.112218,	0.112364,	-0.042145,	-0.12384,	-0.042145,	-0.014689,	-0.014689,	-0.026417,	-0.035438,	-0.061136,	-0.105261,	-0.066309,	-0.08455,	-0.094373,	-0.113074,	-0.109997,	-0.048316,	-0.065701,	-0.005741,	-0.005741,	0.032416,	0.105096,	0.032416,	0.032416]},
    'L9_11' : {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L9_10'   : {'limit':  20, 'gsf': [0.138936,	0.141436,	0.131021,	0.129354,	0.148434,	0.155431,	0.152632,	0.152887,	0.538746,	-0.26047,	0.538746,	-0.069456,	-0.069456,	-0.08998,	-0.105767,	-0.150739,	-0.227957,	-0.15979,	-0.191713,	-0.208902,	-0.241629,	-0.236246,	-0.128303,	-0.158727,	-0.053796,	-0.053796,	0.012979,	0.140168,	0.012979,	0.012979]},
    'L4_12'   : {'limit':  50, 'gsf': [ 0.240763,	0.23591,	0.256131,	0.259366,	0.222326,	0.208741,	0.214175,	0.206478,	0.048164,	-0.035949,	0.048164,	-0.354667,	-0.354667,	-0.308472,	-0.272938,	-0.219042,	-0.090199,	-0.19018,	-0.141277,	-0.114945,	-0.042393,	-0.044234,	-0.1869,	-0.070749,	0.022603,	0.022603,	0.082009,	0.195163,	0.082009,	0.082009]},
    'L12_13' : {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L12_14' : {'limit':  20, 'gsf': [ 0.041722,	0.041141,	0.043561,	0.043949,	0.039515,	0.03789,	0.03854,	0.037461,	0.021313,	0.01263,	0.021313,	0.090144,	0.090144,	-0.449552,	-0.095471,	0.057159,	0.025824,	-0.057721,	-0.035415,	-0.023404,	0.007495,	0.006028,	-0.06127,	-0.015099,	0.0026,	0.0026,	0.013862,	0.035315,	0.013862,	0.013862]},
    'L12_15': {'limit':  20, 'gsf': [ 0.09635,	0.094295,	0.102859,	0.104229,	0.088542,	0.082789,	0.08509,	0.081271,	0.024134,	-0.00659,	0.024134,	0.267688,	0.267688,	-0.103542,	-0.389103,	0.150974,	0.040096,	-0.255527,	-0.176596,	-0.134094,	-0.02476,	-0.029952,	-0.268085,	-0.104709,	-0.042083,	-0.042083,	-0.00223,	0.07368,	-0.00223,	-0.00223]},
    'L12_16': {'limit':  20, 'gsf': [ 0.036024,	0.033807,	0.043044,	0.044522,	0.027602,	0.021396,	0.023878,	0.02108,	-0.06395,	-0.108656,	-0.06395,	0.220834,	0.220834,	0.177955,	0.14497,	-0.493842,	-0.222785,	0.056403,	0.004067,	-0.024114,	-0.091795,	-0.086978,	0.075788,	-0.017608,	-0.00458,	-0.00458,	0.00371,	0.019501,	0.00371,	0.00371]},
    'L14_15': {'limit':  20, 'gsf': [ 0.008389,	0.007808,	0.010228,	0.010615,	0.006182,	0.004556,	0.005207,	0.004127,	-0.01202,	-0.020703,	-0.01202,	0.05681,	0.05681,	0.517115,	-0.128805,	0.023826,	-0.007509,	-0.091055,	-0.068748,	-0.056737,	-0.025838,	-0.027305,	-0.094604,	-0.048432,	-0.030734,	-0.030734,	-0.019471,	0.001982,	-0.019471,	-0.019471]},
    'L16_17': {'limit':  20, 'gsf': [ 0.002691,	0.000474,	0.009711,	0.011189,	-0.005732,	-0.011937,	-0.009455,	-0.012253,	-0.097284,	-0.141989,	-0.097284,	0.187501,	0.187501,	0.144621,	0.111637,	0.472824,	-0.256118,	0.023069,	-0.029266,	-0.057447,	-0.125128,	-0.120311,	0.042455,	-0.050941,	-0.037913,	-0.037913,	-0.029623,	-0.013832,	-0.029623,	-0.029623]},
    'L15_18': {'limit':  50, 'gsf': [ 0.036094,	0.034864,	0.039988,	0.040808,	0.031422,	0.027979,	0.029356,	0.028057,	-0.023588,	-0.0506,	-0.023588,	0.138613,	0.138613,	0.183226,	0.217544,	0.058097,	-0.018394,	-0.526887,	-0.375869,	-0.294552,	-0.034272,	-0.029607,	0.14096,	0.037571,	0.034367,	0.034367,	0.032329,	0.028445,	0.032329,	0.032329]},
    'L18_19': {'limit':  20, 'gsf': [ 0.00276,	0.001531,	0.006655,	0.007475,	-0.001912,	-0.005354,	-0.003977,	-0.005276,	-0.056922,	-0.083934,	-0.056922,	0.10528,	0.10528,	0.149893,	0.184211,	0.024763,	-0.051727,	0.43978,	-0.409203,	-0.327885,	-0.067605,	-0.06294,	0.107627,	0.004238,	0.001034,	0.001034,	-0.001005,	-0.004888,	-0.001005,	-0.001005]},
    'L19_20': {'limit':  20, 'gsf': [ -0.030573,	-0.031803,	-0.026679,	-0.025859,	-0.035245,	-0.038687,	-0.03731,	-0.03861,	-0.090255,	-0.117267,	-0.090255,	0.071946,	0.071946,	0.11656,	0.150878,	-0.00857,	-0.08506,	0.406446,	0.557464,	-0.361219,	-0.100939,	-0.096274,	0.074293,	-0.029095,	-0.032299,	-0.032299,	-0.034338,	-0.038221,	-0.034338,	-0.034338]},
    'L10_20': {'limit':  20, 'gsf': [ 0.063906,	0.065136,	0.060012,	0.059192,	0.068578,	0.072021,	0.070644,	0.071943,	0.123588,	0.1506,	0.123588,	-0.038613,	-0.038613,	-0.083226,	-0.117544,	0.041903,	0.118394,	-0.373113,	-0.524131,	-0.605448,	0.134272,	0.129607,	-0.04096,	0.062429,	0.065633,	0.065633,	0.067671,	0.071555,	0.067671,	0.067671]},
    'L10_17': {'limit':  20, 'gsf': [ 0.030643,	0.03286,	0.023623,	0.022145,	0.039065,	0.045271,	0.042788,	0.045586,	0.130617,	0.175322,	0.130617,	-0.154168,	-0.154168,	-0.111288,	-0.078304,	-0.439491,	-0.710548,	0.010264,	0.0626,	0.09078,	0.158462,	0.153644,	-0.009122,	0.084274,	0.071247,	0.071247,	0.062956,	0.047165,	0.062956,	0.062956]},
    'L10_21': {'limit':  20, 'gsf': [ 0.074932,	0.075233,	0.073979,	0.073778,	0.076075,	0.076918,	0.076581,	0.074271,	0.133441,	0.163049,	0.133441,	0.049841,	0.049841,	0.030768,	0.016097,	0.098015,	0.14378,	0.067413,	0.097737,	0.114065,	-0.506036,	-0.411489,	-0.097141,	-0.250013,	-0.140815,	-0.140815,	-0.071326,	0.061035,	-0.071326,	-0.071326]},
    'L10_22': {'limit':  20, 'gsf': [ 0.040515,	0.040695,	0.039943,	0.039822,	0.041201,	0.041706,	0.041504,	0.040118,	0.07562,	0.093385,	0.07562,	0.02546,	0.02546,	0.014016,	0.005214,	0.054364,	0.081823,	0.036004,	0.054198,	0.063995,	-0.174733,	-0.251338,	-0.062729,	-0.154452,	-0.088934,	-0.088934,	-0.04724,	0.032176,	-0.04724,	-0.04724]},
    'L21_22': {'limit':  20, 'gsf': [ 0.041599,	0.041899,	0.040645,	0.040445,	0.042742,	0.043584,	0.043247,	0.040937,	0.100108,	0.129716,	0.100108,	0.016508,	0.016508,	-0.002565,	-0.017237,	0.064681,	0.110446,	0.03408,	0.064404,	0.080732,	0.46063,	-0.444823,	-0.130475,	-0.283346,	-0.174149,	-0.174149,	-0.104659,	0.027701,	-0.104659,	-0.104659]},
    'L15_23': {'limit':  20, 'gsf': [ 0.035312,	0.033906,	0.039765,	0.040703,	0.029969,	0.026032,	0.027607,	0.024007,	0.002369,	-0.010026,	0.002369,	0.152552,	0.152552,	0.197013,	0.231214,	0.08337,	0.017647,	0.146972,	0.097192,	0.070387,	-0.049659,	-0.060983,	-0.536982,	-0.224046,	-0.140518,	-0.140518,	-0.087363,	0.013883,	-0.087363,	-0.087363]},
    'L22_24': {'limit':  20, 'gsf': [ 0.04878,	0.049261,	0.047255,	0.046934,	0.050609,	0.051957,	0.051418,	0.047722,	0.142395,	0.189767,	0.142395,	0.008635,	0.008635,	-0.021882,	-0.045357,	0.085713,	0.158936,	0.03675,	0.085268,	0.111393,	0.252564,	0.270506,	-0.226538,	-0.471132,	-0.296416,	-0.296416,	-0.185233,	0.026544,	-0.185233,	-0.185233]},
    'L23_24': {'limit':  20, 'gsf': [ 0.001979,	0.000572,	0.006432,	0.00737,	-0.003364,	-0.007301,	-0.005726,	-0.009326,	-0.030964,	-0.043359,	-0.030964,	0.119219,	0.119219,	0.16368,	0.197881,	0.050037,	-0.015687,	0.113638,	0.063859,	0.037054,	-0.082993,	-0.094317,	0.429685,	-0.257379,	-0.173851,	-0.173851,	-0.120697,	-0.01945,	-0.120697,	-0.120697]},
    'L24_25': {'limit':  20, 'gsf': [ 0.017425,	0.0165,	0.020354,	0.02097,	0.013912,	0.011323,	0.012359,	0.005063,	0.078098,	0.113075,	0.078098,	0.09452,	0.09452,	0.108465,	0.119191,	0.102416,	0.109917,	0.117055,	0.115793,	0.115114,	0.136238,	0.142856,	0.169814,	0.238156,	-0.5036,	-0.5036,	-0.339263,	-0.02624,	-0.339263,	-0.339263]},
    'L25_26': {'limit':  20, 'gsf': [ 0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	0.033333,	-0.966667,	0.033333,	0.033333,	0.033333,	0.033333]},
    'L25_27': {'limit':  20, 'gsf': [ -0.049241,	-0.050166,	-0.046313,	-0.045696,	-0.052755,	-0.055344,	-0.054308,	-0.061604,	0.011431,	0.046408,	0.011431,	0.027854,	0.027854,	0.041798,	0.052524,	0.035749,	0.04325,	0.050389,	0.049126,	0.048447,	0.069571,	0.076189,	0.103148,	0.171489,	0.429733,	0.429733,	-0.40593,	-0.092906,	-0.40593,	-0.40593]},
    'L28_27': {'limit':  50, 'gsf': [ 0.149241,	0.150166,	0.146313,	0.145696,	0.152755,	0.155344,	0.154308,	0.161604,	0.088569,	0.053592,	0.088569,	0.072146,	0.072146,	0.058202,	0.047476,	0.064251,	0.05675,	0.049611,	0.050874,	0.051553,	0.030429,	0.023811,	-0.003148,	-0.071489,	-0.329733,	-0.329733,	-0.49407,	0.192906,	-0.49407,	-0.49407]},
    'L27_29': {'limit':  20, 'gsf': [ 0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	0.037415,	-0.676871,	-0.370748]},
    'L27_30': {'limit':  20, 'gsf': [ 0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	0.029252,	-0.256463,	-0.562585]},
    'L29_30': {'limit':  20, 'gsf': [ 0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.004082,	0.289796,	-0.404082]},
    'L8_28': {'limit':  20, 'gsf': [ 0.032071,	0.032255,	0.031485,	0.031362,	0.032773,	0.033291,	0.033084,	0.167876,	0.019936,	0.012941,	0.019936,	0.016651,	0.016651,	0.013863,	0.011717,	0.015072,	0.013572,	0.012145,	0.012397,	0.012533,	0.008308,	0.006984,	0.001593,	-0.012076,	-0.063724,	-0.063724,	-0.096592,	-0.159196,	-0.096592,	-0.096592]},
    'L6_28': {'limit':  20, 'gsf': [ 0.150504,	0.151244,	0.148161,	0.147668,	0.153315,	0.155386,	0.154558,	0.027061,	0.101966,	0.073985,	0.101966,	0.088828,	0.088828,	0.077673,	0.069092,	0.082512,	0.076511,	0.0708,	0.07181,	0.072354,	0.055454,	0.05016,	0.028593,	-0.02608,	-0.232675,	-0.232675,	-0.364145,	-0.614564,	-0.364145,	-0.364145]},
}

# --------------------------------------------------
# 7. HELPER CURVES  (unchanged)
# --------------------------------------------------
def solar_power_from_irradiance(G, G_thr, G_std, P_rated):
    G = np.asarray(G)
    P = np.zeros_like(G, dtype=float)
    mask1 = (G > G_thr) & (G <= G_std)
    mask2 =  G > G_std
    P[mask1] = P_rated * G[mask1] / G_std
    P[mask2] = P_rated
    return P

def wind_power_from_speed(v, v_ci, v_r, v_co, P_rated):
    v = np.asarray(v)
    P = np.zeros_like(v, dtype=float)
    mask_lin = (v >= v_ci) & (v < v_r)
    mask_rtd = (v >= v_r) & (v <= v_co)
    P[mask_lin] = P_rated * (v[mask_lin]**3 - v_ci**3) / (v_r**3 - v_ci**3)
    P[mask_rtd] = P_rated
    return P

# ---------------------------------------------------------------
# 8. INGEST 24-H DATA FROM CSV   ❶ NEW NAMING-SCHEME VERSION
# ---------------------------------------------------------------
CSV_PATH = "all_data_30.csv"          # 24-hour slice
df = pd.read_csv(CSV_PATH)

if len(df) < T:
    raise ValueError(f"CSV only has {len(df)} rows but the model expects T = {T}")

# === ❷ Internal-index → bus-number maps ========================
PV_MG_BUS   = {0: 4,  1: 5,  2: 7,  3: 8,  4: 9,  5: 11}
MGC_BUS     = {0: 1,  1: 2,  2: 3,  3: 6,  4: 13,
               5: 22, 6: 23, 7: 27, 8: 28}
WIND_MG_BUS = {6: 10, 7: 12, 8: 14, 9: 15, 10: 16,
               11: 17, 12: 18, 13: 19, 14: 20, 15: 21,
               16: 24, 17: 25, 18: 26, 19: 29, 20: 30}
# Union of all MGs for loads
MG_BUS = {**PV_MG_BUS, **WIND_MG_BUS}          # ➠ 0…20 ⇒ bus #

# === ❸ Build list of mandatory columns =========================
required_cols = (
    [f"MG{bus}_solar_pred" for bus in PV_MG_BUS.values()]   +
    [f"MGC{bus}_wind_pred" for bus in MGC_BUS.values()]     +
    [f"MG{bus}_wind_pred"  for bus in WIND_MG_BUS.values()] +
    [f"MG{bus}_load_pred"  for bus in MG_BUS.values()]
)
missing = set(required_cols) - set(df.columns)
if missing:
    raise KeyError(f"Missing columns in CSV: {sorted(missing)}")

# === ❹ Slice first 24 rows (0…23) and convert to NumPy =========
# Solar irradiance for PV MGs
G_PV_MG = {
    k: df.loc[:T-1, f"MG{PV_MG_BUS[k]}_solar_pred"].to_numpy()
    for k in PV_MGs
}

# Wind for MGCs
V_WIND_MGC = {
    k: df.loc[:T-1, f"MGC{MGC_BUS[k]}_wind_pred"].to_numpy()
    for k in WIND_MGCs
}

# Wind for MG-7…21 (internal indices 6…20)
V_WIND_MG = {
    k: df.loc[:T-1, f"MG{WIND_MG_BUS[k]}_wind_pred"].to_numpy()
    for k in WIND_MGs
}

# Loads – stacked into shape (NUM_MG, T)
D_MG = np.vstack([
    df.loc[:T-1, f"MG{MG_BUS[k]}_load_pred"].to_numpy()
    for k in range(NUM_MG)
])


In [19]:
# =================================================================
#  SCENARIO GENERATION  &  REDUCTION  – 30-Bus Case
# =================================================================
S_full = 100          # raw scenarios to generate
S_red  = 10           # scenarios to keep after reduction

# --------------------------------------------------
# 1. Deterministic forecast matrices  (shape = T × n_var)
# --------------------------------------------------
# Loads: stack 21 MG demand series column-wise
load_forecast = np.column_stack([D_MG[k] for k in MG])          # (T, 21)

# PV irradiance: six PV-MGs (indices 0-5)
irr_forecast  = np.column_stack([G_PV_MG[k] for k in PV_MGs])   # (T, 6)

# Wind speeds: 9 MGC  + 15 MG  → 24 series
wind_forecast = np.column_stack(
    [V_WIND_MGC[i] for i in WIND_MGCs] +
    [V_WIND_MG[k]  for k in WIND_MGs]
)                                                               # (T, 24)

# --------------------------------------------------
# 2. Generate S_full raw scenarios
# --------------------------------------------------
gen = scen_gen(
    sample_size   = S_full,
    time_period   = T,
    solar_number  = len(PV_MGs),                         # 6
    load_number   = NUM_MG,                              # 21
    wind_number   = len(WIND_MGCs) + len(WIND_MGs)       # 24
)

D_scen = gen.load_scen_gen(load_forecast)       # list[T]  each (S_full, 21)
G_scen = gen.solar_irr_scen_gen(irr_forecast)   # list[T]  each (S_full, 6)
V_scen = gen.wind_speed_scen_gen(wind_forecast) # list[T]  each (S_full, 24)

# Concatenate each hour’s random variables → (S_full, 51)
concat = [
    np.hstack([D_scen[t], G_scen[t], V_scen[t]])
    for t in range(T)
]

In [20]:
# --------------------------------------------------
# 3. Backward scenario-reduction to S_red
# --------------------------------------------------
red = scen_red(
    generated_samples        = concat,
    sample_size              = S_full,
    number_desired_scenarios = S_red,
    time_period              = T,
    load_number              = NUM_MG,                        # 21
    solar_number             = len(PV_MGs),                   # 6
    wind_number              = len(WIND_MGCs) + len(WIND_MGs) # 24
)

# Returns: (list[T] each (S_red, 51)), (S_red,), and component blocks
concat_red, prob_mat, D_red, G_red, V_red = red.reduced_scen()

In [21]:
# =================================================================
#  REBUILD PER-SCENARIO INPUT DICTS   (30-Bus Case)
# =================================================================
# Scenario probabilities (use hour-0 vector, which is already normalised)
p_s = {s: float(prob_mat[0][s]) for s in range(S_red)}

# --- containers -------------------------------------------------
P_WIND_s  = {s: {i: np.empty(T) for i in MGC} for s in range(S_red)}
P_RENEW_s = {s: {mg: np.empty(T) for mg in MG} for s in range(S_red)}
D_s       = {s: np.empty((NUM_MG, T))          for s in range(S_red)}

# --- fill hour-by-hour -----------------------------------------
for t in range(T):
    for s in range(S_red):

        # ........................................................
        # LOAD DEMAND   (MG-0 … MG-20)  → shape (21,)
        # ........................................................
        D_s[s][:, t] = D_red[t][s]          # same ordering as load_forecast

        # ........................................................
        # PV POWER      (6 PV-MGs: indices 0-5)
        # ........................................................
        for pv_idx, mg in enumerate(PV_MGs):           # 0…5
            irr    = G_red[t][s, pv_idx]               # W/m²
            params = PV_PARAMS_MG[mg]
            P_RENEW_s[s][mg][t] = solar_power_from_irradiance(
                irr,
                params['G_thr'],
                params['G_std'],
                params['P_rated']
            )

        # ........................................................
        # WIND POWER
        #   – first 9 speeds  → MGC-0 … MGC-8
        #   – next 15 speeds → MG-6 … MG-20
        # ........................................................
        speeds = V_red[t][s]        # array length 24

        # MGC-side turbines
        for idx, i in enumerate(WIND_MGCs):            # 0…8
            p = WIND_PARAMS_MGC[i]
            P_WIND_s[s][i][t] = wind_power_from_speed(
                speeds[idx],
                p['v_ci'], p['v_r'], p['v_co'], p['P_rated']
            )

        # MG-side wind turbines
        w_offset = len(WIND_MGCs)                      # 9
        for jdx, mg in enumerate(WIND_MGs):           # 15 MGs
            p = WIND_PARAMS_MG[mg]
            P_RENEW_s[s][mg][t] = wind_power_from_speed(
                speeds[w_offset + jdx],
                p['v_ci'], p['v_r'], p['v_co'], p['P_rated']
            )

In [22]:
# =================================================================
# 2. MODEL  –  decision variables (30-bus, 9-MGC, 21-MG)
# =================================================================
m = Model("DA-NMG-two-stage-CVaR-30bus", sense=MAXIMIZE)

# --------------------------------------------------
# 2.1  FIRST-STAGE (scenario-independent) VARIABLES
# --------------------------------------------------
P_mt   = {i: [m.add_var(lb=0,  ub=P_MT_MGC_MAX[i])  for t in range(T)]
          for i in MGC}                                # 9 × 24
u_on   = {i: [m.add_var(var_type=BINARY)          for t in range(T)]
          for i in MGC}
v_st   = {i: [m.add_var(var_type=BINARY)          for t in range(T)]
          for i in MGC}
w_sh   = {i: [m.add_var(var_type=BINARY)          for t in range(T)]
          for i in MGC}
P_grid = {i: [m.add_var(lb=0, ub=P_GRID_MAX[i])   for t in range(T)]
          for i in MGC}
P_exc  = {i: {mg: [m.add_var(lb=0, ub=P_MGC_MAX)  for t in range(T)]
              for mg in MG}
          for i in MGC}                            # 9 × 21 × 24

# --------------------------------------------------
# 2.2  SECOND-STAGE (scenario-dependent) VARIABLES
# --------------------------------------------------
# Redispatch slack (upper-level) – match deterministic UB = 35 MW
G_red = {s: {i: [m.add_var(lb=0, ub=35) for t in range(T)]
             for i in MGC}
         for s in range(S_red)}

# MG-side redispatch slack  (21 × 24)
D_red = {s: [[m.add_var(lb=0, ub=35) for t in range(T)]
             for _ in MG]            # MG is range(NUM_MG)
         for s in range(S_red)}

# ATC & line flows per scenario
ATC  = {s: [m.add_var(lb=0) for t in range(T)] for s in range(S_red)}
Flow = {s: {ln: [m.add_var(lb=-info['limit'], ub=info['limit'])
                 for t in range(T)]
            for ln, info in LINES.items()}
        for s in range(S_red)}

# Local MG gas-turbine dispatch  (2 GTs per MG)
P_mg = {s: {mg: {g: [m.add_var(lb=P_MG_MIN[mg][g],
                              ub=P_MG_MAX[mg][g])
                    for t in range(T)]
                 for g in range(2)}
            for mg in MG}
        for s in range(S_red)}

# --------------------------------------------------
# 2.2.a  BATTERY VARIABLES  (BATT_MG = [0…5])
# --------------------------------------------------
P_chr, P_dis, E = {}, {}, {}
alpha_soc_min, alpha_soc_max = {}, {}
beta_chr_min,  beta_chr_max  = {}, {}
beta_dis_min,  beta_dis_max  = {}, {}
z_soc_min, z_soc_max = {}, {}
z_chr_min, z_chr_max = {}, {}
z_dis_min, z_dis_max = {}, {}
gamma_end, z_end = {}, {}
phi = {}

def batt_idx(mg: int) -> int:
    """Map MG index → position in battery parameter arrays (0-5)."""
    return BATT_MG.index(mg)

for s in range(S_red):
    for mg in BATT_MG:                          # six battery MGs
        k = batt_idx(mg)                        # 0…5
        # Power & energy variables
        P_chr[s, mg] = [m.add_var(lb=0, ub=P_chr_max[k]) for t in range(T)]
        P_dis[s, mg] = [m.add_var(lb=0, ub=P_dis_max[k]) for t in range(T)]
        E[s, mg]     = [m.add_var(lb=SOC_min[k]*E_R[k],
                                  ub=SOC_max[k]*E_R[k])  for t in range(T)]

        # Duals & binaries for KKT limits
        alpha_soc_min[s, mg] = [m.add_var(lb=0)            for t in range(T)]
        alpha_soc_max[s, mg] = [m.add_var(lb=0)            for t in range(T)]
        z_soc_min[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]
        z_soc_max[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]

        beta_chr_min[s, mg]  = [m.add_var(lb=0)            for t in range(T)]
        beta_chr_max[s, mg]  = [m.add_var(lb=0)            for t in range(T)]
        z_chr_min[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]
        z_chr_max[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]

        beta_dis_min[s, mg]  = [m.add_var(lb=0)            for t in range(T)]
        beta_dis_max[s, mg]  = [m.add_var(lb=0)            for t in range(T)]
        z_dis_min[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]
        z_dis_max[s, mg]     = [m.add_var(var_type=BINARY) for t in range(T)]

        gamma_end[s, mg] = m.add_var(lb=0)
        z_end[s, mg]     = m.add_var(var_type=BINARY)
        phi[s, mg]       = [m.add_var() for t in range(T)]

# --------------------------------------------------
# 2.3  KKT DUALS ON MG TURBINES
# --------------------------------------------------
def dual_array(binaries: bool = False):
    """Return NUM_MG × 2 × T nested list of dual (or binary) vars."""
    return [[[m.add_var(var_type=BINARY) if binaries else m.add_var(lb=0)
              for t in range(T)]
             for _ in range(2)]
            for _ in range(NUM_MG)]             # NUM_MG = 21

alpha_min, alpha_max = {}, {}
beta_up,   beta_dn   = {}, {}
z_min, z_max         = {}, {}
z_up,  z_dn          = {}, {}
lam                 = {}

for s in range(S_red):
    alpha_min[s] = dual_array()
    alpha_max[s] = dual_array()
    beta_up[s]   = dual_array()
    beta_dn[s]   = dual_array()
    z_min[s]     = dual_array(binaries=True)
    z_max[s]     = dual_array(binaries=True)
    z_up[s]      = dual_array(binaries=True)
    z_dn[s]      = dual_array(binaries=True)
    lam[s]       = [[m.add_var() for t in range(T)] for _ in MG]

# --------------------------------------------------
# 3.  CVaR VARIABLES  &  RISK PARAMETERS
# --------------------------------------------------
Z     = {s: m.add_var()  for s in range(S_red)}    # total cost for scenario-s
zeta  = m.add_var()                                # VaR proxy
xi    = {s: m.add_var(lb=0) for s in range(S_red)}

alpha_cvar  = 0.95         # confidence level
lambda_risk = 0.5          # 0 = risk-neutral
c_coeff     = 1.0 / (1.0 - alpha_cvar)

In [23]:
# =================================================================
# 4.  CONSTRAINTS   (30-Bus upgrade)
# =================================================================
# -----------------------------------------------------------------
# 4.1  MGC commitment, ramps, min-up / min-down
# -----------------------------------------------------------------
for i in MGC:

    # Unit limits & logical start/stop links
    for t in range(T):
        m += P_mt[i][t] <= P_MT_MGC_MAX[i] * u_on[i][t]
        m += P_mt[i][t] >= P_MT_MGC_MIN[i] * u_on[i][t]

        if t == 0:                                   # assumed off at t=0
            m += v_st[i][t] - w_sh[i][t] == u_on[i][t]
            m += w_sh[i][t] == 0
        else:
            m += v_st[i][t] - w_sh[i][t] == u_on[i][t] - u_on[i][t-1]

    # Ramps
    for t in range(1, T):
        m +=  P_mt[i][t]   - P_mt[i][t-1] <= RU_MGC[i]
        m +=  P_mt[i][t-1] - P_mt[i][t]   <= RD_MGC[i]

    # Minimum up / down
    for t in range(T - MIN_UP[i]):
        m += xsum(u_on[i][k] for k in range(t+1, t+1+MIN_UP[i])) \
             >= MIN_UP[i] * v_st[i][t+1]

    for t in range(T - MIN_DOWN[i]):
        m += xsum(1 - u_on[i][k] for k in range(t+1, t+1+MIN_DOWN[i])) \
             >= MIN_DOWN[i] * w_sh[i][t+1]

# -----------------------------------------------------------------
# 4.2  MGC → MG export pipe  (shared cap per MG)
# -----------------------------------------------------------------
for mg in MG:
    for t in range(T):
        m += xsum(P_exc[i][mg][t] for i in MGC) <= P_MGC_MAX   # 35 MW

# -----------------------------------------------------------------
# 4.3  Scenario-dependent constraints  &  ATC / Line-flows
# -----------------------------------------------------------------
for s in range(S_red):

    Pwind = P_WIND_s[s]     # dict{i}[t]
    Dem   = D_s[s]          # (21, T)
    Pren  = P_RENEW_s[s]    # dict{mg}[t]

    # (i)  MGC power balances
    for i in MGC:
        for t in range(T):
            m += (
                P_mt[i][t] + Pwind[i][t] + P_grid[i][t]
                >= xsum(P_exc[i][mg][t] for mg in MG)
            )

    # (ii)  MG power balances (inc. batteries where present)
    for mg in MG:
        for t in range(T):
            batt = 0.0
            if mg in BATT_MG:
                batt = P_dis[s, mg][t] - P_chr[s, mg][t]

            m += (
                P_mg[s][mg][0][t] + P_mg[s][mg][1][t]
                + Pren[mg][t] + batt
                + xsum(P_exc[i][mg][t] for i in MGC)
                >= Dem[mg][t]
            )

    # (iii)  Redispatch balance, ATC & line-flows
    for t in range(T):

        # --- system-wide redispatch ---------------------------
        expr_red = xsum(
            G_red[s][i][t] - (P_mt[i][t] + P_grid[i][t] + Pwind[i][t])
            for i in MGC
        )

        m += expr_red == xsum(
            D_red[s][mg][t] - xsum(P_exc[i][mg][t] for i in MGC)
            for mg in MG
        )
        m += ATC[s][t] == expr_red

        # Non-negativity of slack
        for i in MGC:
            m += G_red[s][i][t] >= P_mt[i][t] + P_grid[i][t] + Pwind[i][t]
        for mg in MG:
            m += D_red[s][mg][t] >= xsum(P_exc[i][mg][t] for i in MGC)

        # --- PTDF line-flow definitions (length-30 vectors) ---
        for ln, info in LINES.items():
            gsf = info["gsf"]

            flow_expr = (
                gsf[0]  * G_red[s][0][t]   +  # Bus 1  (MGC-0)
                gsf[1]  * G_red[s][1][t]   +  # Bus 2  (MGC-1)
                gsf[2]  * G_red[s][2][t]   +  # Bus 3  (MGC-2)
                gsf[3]  * D_red[s][0][t]   +  # Bus 4  (MG-0)
                gsf[4]  * D_red[s][1][t]   +  # Bus 5  (MG-1)
                gsf[5]  * G_red[s][3][t]   +  # Bus 6  (MGC-3)
                gsf[6]  * D_red[s][2][t]   +  # Bus 7  (MG-2)
                gsf[7]  * D_red[s][3][t]   +  # Bus 8  (MG-3)
                gsf[8]  * D_red[s][4][t]   +  # Bus 9  (MG-4)
                gsf[9]  * D_red[s][14][t]  +  # Bus 10 (MG-14)
                gsf[10] * D_red[s][5][t]   +  # Bus 11 (MG-5)
                gsf[11] * D_red[s][6][t]   +  # Bus 12 (MG-6)
                gsf[12] * G_red[s][5][t]   +  # Bus 13 (MGC-5)
                gsf[13] * D_red[s][7][t]   +  # Bus 14 (MG-7)
                gsf[14] * D_red[s][8][t]   +  # Bus 15 (MG-8)
                gsf[15] * D_red[s][9][t]   +  # Bus 16 (MG-9)
                gsf[16] * D_red[s][10][t]  +  # Bus 17 (MG-10)
                gsf[17] * D_red[s][11][t]  +  # Bus 18 (MG-11)
                gsf[18] * D_red[s][12][t]  +  # Bus 19 (MG-12)
                gsf[19] * D_red[s][13][t]  +  # Bus 20 (MG-13)
                gsf[20] * D_red[s][15][t]  +  # Bus 21 (MG-15)
                gsf[21] * G_red[s][7][t]   +  # Bus 22 (MGC-7)
                gsf[22] * G_red[s][6][t]   +  # Bus 23 (MGC-6)
                gsf[23] * D_red[s][16][t]  +  # Bus 24 (MG-16)
                gsf[24] * D_red[s][17][t]  +  # Bus 25 (MG-17)
                gsf[25] * D_red[s][18][t]  +  # Bus 26 (MG-18)
                gsf[26] * G_red[s][8][t]   +  # Bus 27 (MGC-8)
                gsf[27] * G_red[s][4][t]   +  # Bus 28 (MGC-4)
                gsf[28] * D_red[s][19][t]  +  # Bus 29 (MG-19)
                gsf[29] * D_red[s][20][t]     # Bus 30 (MG-20)
            )
            m += Flow[s][ln][t] == flow_expr

    # =================================================================
    # (iv)  Lower-level KKT  – MG gas turbines
    # =================================================================
    for mg in MG:
        for t in range(T):
            for g in range(2):

                # Bounds & ramps
                m += P_mg[s][mg][g][t] >= P_MG_MIN[mg][g]
                m += P_mg[s][mg][g][t] <= P_MG_MAX[mg][g]

                if t > 0:
                    m += P_mg[s][mg][g][t]   - P_mg[s][mg][g][t-1] <= RU_MG
                    m += P_mg[s][mg][g][t-1] - P_mg[s][mg][g][t]   <= RD_MG

                # Complementarity (Big-M)
                m += alpha_min[s][mg][g][t] <= M * z_min[s][mg][g][t]
                m += (P_MG_MIN[mg][g] - P_mg[s][mg][g][t]) <= M * (1 - z_min[s][mg][g][t])

                m += alpha_max[s][mg][g][t] <= M * z_max[s][mg][g][t]
                m += (P_mg[s][mg][g][t] - P_MG_MAX[mg][g]) <= M * (1 - z_max[s][mg][g][t])

                if t > 0:
                    m += beta_up[s][mg][g][t] <= M * z_up[s][mg][g][t]
                    m += (P_mg[s][mg][g][t] - P_mg[s][mg][g][t-1] - RU_MG) \
                         <= M * (1 - z_up[s][mg][g][t])

                    m += beta_dn[s][mg][g][t] <= M * z_dn[s][mg][g][t]
                    m += (P_mg[s][mg][g][t-1] - P_mg[s][mg][g][t] - RD_MG) \
                         <= M * (1 - z_dn[s][mg][g][t])

            # ---- Stationarity wrt GT output -------------------
            for g in range(2):

                if T == 1:
                    delta = 0
                elif t == 0:
                    delta = -beta_up[s][mg][g][t+1] + beta_dn[s][mg][g][t+1]
                elif t == T - 1:
                    delta =  beta_up[s][mg][g][t]   - beta_dn[s][mg][g][t]
                else:
                    delta = ( beta_up[s][mg][g][t]   - beta_dn[s][mg][g][t]
                            - beta_up[s][mg][g][t+1] + beta_dn[s][mg][g][t+1])

                # Avoid divide-by-zero for wind-only MGs
                cost_term = 0 if ETA_MG[mg][g] == 0 else \
                            C_NG / (L_NG * ETA_MG[mg][g])

                m += (cost_term - lam[s][mg][t]
                      - alpha_min[s][mg][g][t] + alpha_max[s][mg][g][t]
                      + delta == 0)

    # =================================================================
    # (v)  Battery KKT   – MGs with storage
    # =================================================================
    for mg in BATT_MG:
        k = batt_idx(mg)

        # Initial / terminal SOC
        m += E[s, mg][0]   == E_init[k]
        m += E[s, mg][T-1] >= E_end[k]

        m += gamma_end[s, mg] <= M * z_end[s, mg]
        m += (E[s, mg][T-1] - E_end[k]) <= M * (1 - z_end[s, mg])

        # Energy dynamics
        for t in range(T-1):
            m += (E[s, mg][t+1] == E[s, mg][t]
                  + ETA_CHR[k] * P_chr[s, mg][t] * Delta_t
                  - P_dis[s, mg][t] * Delta_t / ETA_DIS[k])

        # φ stationarity chain
        for t in range(T-1):
            m += (phi[s, mg][t] - phi[s, mg][t+1]
                  + alpha_soc_max[s, mg][t] - alpha_soc_min[s, mg][t] == 0)

        m += (phi[s, mg][T-1] + gamma_end[s, mg]
              + alpha_soc_max[s, mg][T-1] - alpha_soc_min[s, mg][T-1]
              - 45 == 0)

        # Bounds & complementarity
        for t in range(T):

            # SOC bounds
            m += alpha_soc_min[s, mg][t] <= M * z_soc_min[s, mg][t]
            m += (E[s, mg][t] - SOC_min[k]*E_R[k]) <= M * (1 - z_soc_min[s, mg][t])

            m += alpha_soc_max[s, mg][t] <= M * z_soc_max[s, mg][t]
            m += (SOC_max[k]*E_R[k] - E[s, mg][t]) <= M * (1 - z_soc_max[s, mg][t])

            # Charge bounds
            m += beta_chr_min[s, mg][t] <= M * z_chr_min[s, mg][t]
            m += P_chr[s, mg][t] <= M * (1 - z_chr_min[s, mg][t])

            m += beta_chr_max[s, mg][t] <= M * z_chr_max[s, mg][t]
            m += (P_chr[s, mg][t] - P_chr_max[k]) <= M * (1 - z_chr_max[s, mg][t])

            # Discharge bounds
            m += beta_dis_min[s, mg][t] <= M * z_dis_min[s, mg][t]
            m += P_dis[s, mg][t] <= M * (1 - z_dis_min[s, mg][t])

            m += beta_dis_max[s, mg][t] <= M * z_dis_max[s, mg][t]
            m += (P_dis[s, mg][t] - P_dis_max[k]) <= M * (1 - z_dis_max[s, mg][t])

            # Stationarity wrt P_chr / P_dis
            phi_next = phi[s, mg][t+1] if t < T-1 else gamma_end[s, mg]

            m += (K_CHR * Delta_t
                  + lam[s][mg][t] * ETA_CHR[k] * Delta_t
                  - phi_next
                  - beta_chr_min[s, mg][t] + beta_chr_max[s, mg][t] == 0)

            m += (K_DIS * Delta_t
                  - lam[s][mg][t] * Delta_t / ETA_DIS[k]
                  + phi_next
                  - beta_dis_min[s, mg][t] + beta_dis_max[s, mg][t] == 0)

    # =================================================================
    # (vi)  Scenario-cost definition (profit style, MAX objective)
    # =================================================================
    scen_profit = (
        # Revenue: MGC→MG exports
        xsum(C_EXC[i][t] * P_exc[i][mg][t]
             for i in MGC for mg in MG for t in range(T))
        # Redispatch / ATC incentive
      + xsum(LAMBDA_ATC * ATC[s][t] for t in range(T))
        # Cost terms (reduce profit)
      - xsum(
            (C_NG_MGC[i] / (L_NG_MGC[i] * ETA_MGC[i])) * P_mt[i][t]
            + STARTUP_COST[i]   * v_st[i][t]
            + SHUTDOWN_COST[i]  * w_sh[i][t]
            + C_GRID[t]         * P_grid[i][t]
            + C_WIND[i]         * Pwind[i][t]     # wind curtail/cost
            for i in MGC for t in range(T)
        )
    )

    # Bind scenario value
    m += Z[s] == scen_profit

    # CVaR auxiliary
    m += xi[s] >= zeta - Z[s]

In [24]:
# =================================================================
# 5. OBJECTIVE  —  (1−λ)·E[Profit]  +  λ·CVaRα
# =================================================================
obj_expect = xsum(p_s[s] * Z[s]            # expected profit across scenarios
                  for s in range(S_red))

obj_cvar   = zeta - c_coeff * xsum(p_s[s] * xi[s]
                                   for s in range(S_red))

m.objective = (1 - lambda_risk) * obj_expect + lambda_risk * obj_cvar
# (Model sense already MAXIMIZE.)

# =================================================================
# 6. SOLVE
# =================================================================
m.optimize()     # add  m.optimize(max_seconds=900)  if you need a hard cap

# =================================================================
# 7. REPORT  –  summary + representative scenario (s = 0)
# =================================================================
if m.num_solutions:

    exp_profit = sum(p_s[s] * Z[s].x for s in range(S_red))
    cvar_tail  = zeta.x - c_coeff * sum(p_s[s] * xi[s].x for s in range(S_red))

    print("\n============= SUMMARY (30-Bus CVaR) =============")
    print(f"✓ Expected profit                 = {exp_profit:,.2f}")
    print(f"✓ CVaR (α = {alpha_cvar:.0%})            = {cvar_tail:,.2f}")
    print(f"✓ Combined objective (λ = {lambda_risk}) = {m.objective_value:,.2f}")
    print("=================================================\n")

    # ------------ display one scenario in detail ---------------
    rep_s = 0                       # pick a scenario to inspect
    watch_lines = ['L4_12', 'L6_10', 'L9_10',
                   'L28_27', 'L10_17', 'L10_20', 'L23_24']  # tie-lines of interest
    line_names = [ln for ln in watch_lines if ln in LINES]

    print(f"--- Scenario {rep_s} dispatch details ---")
    for t in range(T):
        print(f"\nHour {t:02d}")

        # ---------- MGCs ---------------------------------------
        for i in MGC:
            print(f"  MGC{i+1:>2}: Grid={P_grid[i][t].x:7.2f}  "
                  f"MT={P_mt[i][t].x:7.2f}  "
                  f"Wind={P_WIND_s[rep_s][i][t]:6.2f}  "
                  f"On={int(u_on[i][t].x)}")

        # ---------- Selected network flows --------------------
        flows_str = "  ".join(
            f"Flow({ln})={Flow[rep_s][ln][t].x:8.2f}" for ln in line_names
        )
        print(f"  ATC={ATC[rep_s][t].x:7.2f}  {flows_str}")

        # ---------- Microgrids --------------------------------
        for mg in MG:
            gens  = [P_mg[rep_s][mg][g][t].x for g in range(2)]
            renew = P_RENEW_s[rep_s][mg][t]
            excs  = [P_exc[i][mg][t].x for i in MGC]

            batt_chr = batt_dis = soc = 0.0
            if mg in BATT_MG:
                batt_chr = P_chr[rep_s, mg][t].x
                batt_dis = P_dis[rep_s, mg][t].x
                soc      = E[rep_s, mg][t].x

            batt_net = batt_dis - batt_chr
            total    = sum(gens) + renew + batt_net + sum(excs)
            redisp   = D_red[rep_s][mg][t].x

            line = (f"  MG{mg+1:>2}: Dem={D_s[rep_s][mg][t]:6.2f}  "
                    f"GTs={np.round(gens,2)}  Ren={renew:6.2f}  ")

            if mg in BATT_MG:
                line += (f"Chr={batt_chr:6.2f}  Dis={batt_dis:6.2f}  "
                         f"E={soc:6.2f}  ")

            line += (f"Exc={np.round(excs,2)}  Tot={total:8.2f}  "
                     f"Redisp={redisp:6.2f}")
            print(line)
else:
    print("❌ No feasible solution found.")


============= SUMMARY (30-Bus CVaR) =============
✓ Expected profit                 = 95,056.26
✓ CVaR (α = 95%)            = 95,050.55
✓ Combined objective (λ = 0.5) = 95,053.40

--- Scenario 0 dispatch details ---

Hour 00
  MGC 1: Grid=  20.00  MT=   6.00  Wind=  4.58  On=1
  MGC 2: Grid=  20.00  MT=   6.00  Wind=  4.56  On=1
  MGC 3: Grid=  20.00  MT=   5.50  Wind=  3.58  On=1
  MGC 4: Grid=  20.00  MT=   5.00  Wind=  5.33  On=1
  MGC 5: Grid=  20.00  MT=   6.50  Wind=  3.40  On=1
  MGC 6: Grid=  20.00  MT=   5.50  Wind=  5.57  On=1
  MGC 7: Grid=  20.00  MT=   5.50  Wind=  5.08  On=1
  MGC 8: Grid=  20.00  MT=   5.00  Wind=  3.76  On=1
  MGC 9: Grid=  20.00  MT=   5.00  Wind=  4.04  On=1
  ATC=  45.11  Flow(L4_12)=   22.44  Flow(L6_10)=    4.95  Flow(L9_10)=    0.95  Flow(L28_27)=   -9.05  Flow(L10_17)=   20.00  Flow(L10_20)=   20.00  Flow(L23_24)=   -5.85
  MG 1: Dem= 16.62  GTs=[6. 6.]  Ren=  0.00  Chr=  0.06  Dis=  0.00  E=  0.07  Exc=[0.   0.   0.   0.   0.   0.   0.   4.51 0